# Dashboard

This pipeline processes data into a form to drive the dashboard.

The parameters are injected using papermill.

In [117]:
target: str = 'q1'
start: str = '2025-01-01'
end: str = '2025-03-31'
fixed_date: str = 'true'

## Setup

Import some libraries

In [118]:
import json
from datetime import date
from ast import literal_eval

import petl as etl
from utils.paths import PUBLISHED, SITE

Set the target directory, and ensure it exists

In [119]:
TARGET = SITE / 'insights/dashboard' / target / '_data'
TARGET.mkdir(exist_ok=True, parents=True)

Calculate the date range. If the `fixed_date` parameter is `false`, make today the latest date.

In [120]:
date_range = (
    date.fromisoformat(start),
    min(date.today(), date.fromisoformat(end)) if fixed_date == 'false' else date.fromisoformat(end),
    date.today(),
)

Write the timestamp file

In [121]:
with open(TARGET / 'timestamp.json', 'w') as f:
    json.dump(dict(zip(['start', 'end', 'updated'], (str(x) for x in date_range))), f)

## Projects data

In [122]:
projects = (
    etl
    .fromjson(PUBLISHED / 'programme/projects.json')
    .convert(['Start Date', 'End Date'], etl.dateparser('%Y-%m-%d'))
    .convert('Project Name', lambda x: x.strip())
    .selectge('End Date', date_range[0])
    .selectle('Start Date', date_range[1])
)

In [123]:
def category_exploder(row):
    for v in row['Producing model']:
        yield [v]

producing_model = etl.cat(
    projects
    .rowmapmany(category_exploder, header=['Producing model'])
    .aggregate('Producing model', len),

    projects
    .selecteq('Producing model', [])
    .aggregate(None, len)
    .addfield('Producing model', 'UNKNOWN')
)

In [124]:
project_summary = {
    'count': projects.nrows(),
    'producing_model': dict(producing_model.records()),
    'names': list(projects.cut('Project Name').distinct().values('Project Name')),
}

In [125]:
events = (
    etl
    .fromcsv(PUBLISHED / 'programme/events.csv')
    .convert(['Start Date', 'End Date'], etl.dateparser('%Y-%m-%d'))
    .convert('Programme Category', lambda f: set(literal_eval(f)))
    .convert('Project Name', lambda x: x.strip())
    .replaceall('', None)
    .selectge('End Date', date_range[0])
    .selectle('Start Date', date_range[1])
    .selectin('Item Type', ['Event (any public-facing activity)'])
    .selectnotnone('Project Name')

    .cut('Start Date', 'End Date', 'project_id', 'Project Name', 'Programme Category')
    .convert('Start Date', lambda d: max(d, date_range[0]))
    .convert('End Date', lambda d: min(d, date_range[1]))
    .addfield('Events', lambda r: max(1, (r['End Date'] - r['Start Date']).days + 1))

    .sort(['End Date', 'Start Date'])
)

In [126]:
manual = (
    etl
    .fromcsv(PUBLISHED / 'manual/manual-events.csv')
    .selectne('Exclude from events count', 'True')
    .convert('Date', lambda d: date.fromisoformat(d))
    .selectge('Date', date_range[0])
    .selectle('Date', date_range[1])
    .convertnumbers()
    .update('Events', 1, where=lambda r: not r.Events)
    .aggregate(['Project', 'Airtable project ID'], sum, 'Events')
    .rename({
        'Project': 'Project Name',
        'Airtable project ID': 'project_id'
    })
)

In [127]:
combined_events = (
    etl.cat(
        events.aggregate(['project_id', 'Project Name'], sum, 'Events').selectnotin('project_id', [i for i in manual.values('project_id') if i != '']),
        manual
    )
    .cutout('project_id')
    .convert('Project Name', lambda n: n.strip())
    .convert('Project Name', {
        'Rise (AKA - Opening Event)': 'RISE',
        'Our Patch (formerly Magic Waiting)': 'Our Patch',
    })
    .aggregate('Project Name', sum, 'value')
)

In [128]:
def category_exploder(row):
    for c in row['Programme Category']:
        yield (row['Project Name'], c)

events_summary = {
    'count': events.nrows(),
    # TODO Update this when there is a way of filtering by in-person / online events
    'breakdown': {
        'in_person': events.nrows(),
        'digital': 0,
    },
    'projects': dict(combined_events.records()),
    'categories': dict(events.cut('Project Name', 'Programme Category').rowmapmany(category_exploder, header=['project', 'category']).aggregate(['category'], len).records())
}

In [129]:
with open(TARGET / 'events.json', 'w') as f:
    json.dump(
        {
            'projects': project_summary,
            'events': events_summary,
        },
        f,
        indent=2
    )

## Tickets

In [130]:
ticketed_events = (
    etl
    .fromcsv(PUBLISHED / 'ticketing/event-instances.csv')
    .convert('start', etl.dateparser('%Y-%m-%d %H:%M:%S'))
    .selectrangeopen('start', date_range[0], date_range[1])
    .selectcontains('eventType', 'MainProgrammeEvent')
)

In [131]:
tickets_sold = (
    etl
    .fromcsv(PUBLISHED / 'ticketing/tickets.csv')
    .selecteq('geography_type', 'oslaua')
    .rightjoin(ticketed_events.cut('instance_id', 'event_name'))
    .cut('event_name', 'geography_code', 'count_of_tickets')
    .convertnumbers()
    .selectnotnone('count_of_tickets')
)

In [132]:
tickets_scanned = dict(
    etl
    .fromcsv(PUBLISHED / 'manual/spektrix-sales-and-scans.csv')
    .convert('Event Date', etl.dateparser('%Y-%m-%d'))
    .selectrangeopen('Event Date', date_range[0], date_range[1])
    .convertnumbers()
    .aggregate(None, {
        'sold': ('Sold', sum),
        'scanned': ('Scanned', sum)
    })
    .melt(variables=['sold', 'scanned'])
    .records()
)

In [133]:
ticket_summary = {
    'events': {
        'names': list(ticketed_events.cut('event_name').distinct().values('event_name')),
        'count': ticketed_events.nrows(),
    },
    'sold': {
        'total': sum(tickets_sold.values('count_of_tickets')),
        'bradford': sum(tickets_sold.selecteq('geography_code', 'E08000032').values('count_of_tickets')),
    },
    'manual': tickets_scanned
}

In [134]:
with open(TARGET / 'tickets.json', 'w') as f:
    json.dump(
        ticket_summary,
        f,
        indent=2
    )

## Volunteers

In [135]:
signups = (
    etl
    .fromcsv(PUBLISHED / 'volunteers/checkpoints.csv')
    .convert('date', etl.dateparser('%Y-%m-%d'))
    .selectle('date', date_range[1])
    .convertnumbers(strict=False)
)

In [136]:
shifts = (
    etl
    .fromcsv(PUBLISHED / 'volunteers/shifts.csv')
    .convert('date', etl.dateparser('%Y-%m-%d'))
    .selecteq('type', 'BD25 Event')
    .selectrangeopen('date', date_range[0], date_range[1])
    .convertnumbers(strict=False)
)

In [137]:
volunteer_summary = {
    'events': {
        'count': shifts.cut('rosterfy_event_name').distinct().nrows(),
        'names': list(shifts.cut('rosterfy_event_name').distinct().values('rosterfy_event_name')),
    },
    'shifts': {
        'count': shifts.nrows(),
        'attended': sum(shifts.values('attended')),
        'hours': sum(shifts.values('hours')),
    },
    'people': {
        'checkpoints': dict(signups.aggregate('checkpoint', sum, 'count').records()),
    },
}

In [138]:
with open(TARGET / 'volunteers.json', 'w') as f:
    json.dump(
        volunteer_summary,
        f,
        indent=2
    )